In [2]:
import networkx as nx
from networkx.algorithms import community #  
import itertools
import matplotlib.pyplot as plt
import csv
from networkx.algorithms.community.quality import modularity
import random
import wikipediaapi
from keybert import KeyBERT
import time
from gensim import models,corpora
from math import log, exp

d:\anaconda\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def ReadTweetsByTime(dataname,date):
    doclist=[]
    if dataname=='GPT': #搜集的推特关于GPT的数据集，时间从一月四号持续到三月29号
       with open('../推特GPT/GPT.csv',encoding="utf-8") as f:
         f_csv = csv.reader(f)
         pos = 1
         for row in f_csv:
            if  len(row)>=6:  #控制聚类文本数目并处理缺失数据,全部跑太多了跑不了
                #print(row[0][:10])  #取到日期
                if row[0][:10] == date: #判断时间
                    #print(row[0])
                    doclist.append(row[2])
                    pos+=1
         print(f'总数量{pos}')
    #return random.sample(doclist,200)
    global data_number
    if data_number!=0:
        return random.sample(doclist,data_number)
    else:  #表示数据全部要
        data_number=len(doclist)
        return doclist

In [ ]:
#工具函数算边
def GetBrim(list_word):
    #获取节点间的边信息，主要使用相似度
    list_link_dic=[]
    global threshold_similarity  #全局变量，边的门限
    model = models.word2vec.Word2Vec.load('word2vec.model')  #使用相似度试试
    for i in range(0,len(list_word)):
        dic={}
        for j in range(i+1,len(list_word)):
            try:
               similarity = model.wv.similarity(list_word[i],list_word[j])
            except:
               similarity=0   #主要是不存在某个词的情况
            if(similarity>=threshold_similarity):  #判断点的权重
                dic={}
                dic['Source']=i
                dic['Target']=j
                dic['Type']=0
                dic['Weight']=similarity
                list_link_dic.append(dic)
        #print(f"\r算边中:{(i+1)/len(list_word)}",end=' ')
        print("\r", end="")
        print(f'算边进度为{(i+1)/len(list_word)}',end="",flush=True)
    print('\n完成算边')
    return list_link_dic

In [4]:
def GetKeyBertWord(dataname,date):
    #使用keybert方法，获取关键词
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    doclist=ReadTweetsByTime(dataname,date)
    dic={}
    list_word=[]
    list_weight=[]
    pos=0
    for item in doclist:
        #print(item)
        pos=pos+1
        global data_number
        #print(pos/data_number)
        print("\r", end="")
        print("取词中: {}".format(pos/data_number),end="") #进度条
        #有点慢，两千条数据要跑十分钟
        keywords_phrase = kw_model.extract_keywords(item, 
                                     keyphrase_ngram_range=(1,1),   #这个就是范围,可以考虑词组,本实验暂时只考虑词
                                     stop_words='english', 
                                     highlight=False, 
                                     top_n=3) 
        keywords_phrase_list= list(keywords_phrase)#结果，返回的是列表包含元组的形式[(),()]
        #print(keywords_phrase_list)
        
        for word in keywords_phrase_list:
            #print(word[0])
            #把数据加入字典,要处理1.去除全局的词  2.判断是否已经存在于字典中
            if word[0] not in ['chatgpt','gpt','#chatgpt','chat']:
              if word[0] not in list_word:
                list_word.append(word[0])
                list_weight.append(word[1])
              else:
                index=list_word.index(word[0])
                list_weight[index]=list_weight[index]+word[1]
        #print(keywords_list)
        
    #最终返回两个数据，字典形式的结果和原始数据
    dic['word']=list_word
    dic['weight']=list_weight
    print(' ')#这个空行主要是为了和进度条配合完成换行,后面也有
    return dic,doclist

In [5]:
def ReadNodeAndList(file):
    #从文件中读取节点和边
    list_node=[]
    list_link=[]
    list_node_label=[]
    list_size=[]
    file_node=file+'-node.csv'
    with open(file_node, encoding="UTF8") as f:
        reader = csv.reader(f)
        header_row = next(reader)
        for row in reader:
            #print(row)
            list_node.append(row[0])
            list_node_label.append(row[1])
            list_size.append(row[4])
    file_link=file+'-link.csv'
    with open(file_link, encoding="UTF8") as f:
        reader = csv.reader(f)
        header_row = next(reader)
        for row in reader:
            #print(row)
            tuple=(row[0],row[1],eval(row[3])) #依次为source target weight  注意是无向的
            #print(type(eval(row[3])))    #要转数字形式
            list_link.append(tuple)
    return list_node,list_link,list_node_label,list_size

In [6]:
def typeLabel(list_word):
    #获得话题的label，方法为从wiki搜索指定几个关键词的网页并解析，然后选取最关键词
    #注意连vpn
    #user_agent需要时常修改
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.60'
    #print('成功加载模型')
    def get_word_background(word):
        wiki = wikipediaapi.Wikipedia(language='en',user_agent=user_agent)
        page = wiki.page(word)
        if page.exists():
            return page.summary
        else:
            return ''
    page_source=''
    for item in list_word:
        page_source = page_source+get_word_background(item)
    keywords_phrase = kw_model.extract_keywords(page_source, 
                                     keyphrase_ngram_range=(1,1),   #这个就是范围,可以考虑词组
                                     stop_words='english', 
                                     highlight=False, 
                                     top_n=1) 
    keywords_phrase_list= list(keywords_phrase)
    for word in keywords_phrase_list:
        return word[0]

In [7]:
def writeTypeNode(filename,list_dic):
    #生成带type和typelabel的csv文件
    #这一步主要是为了gephi画图。  在gephi中打开node_type和link即可
    filename=filename+'-node_type.csv'
    with open(filename, 'w', encoding='UTF8',newline="") as f_link:
        writer = csv.DictWriter(f_link,fieldnames=['id','label','size','type','typelabel']) 
        writer.writeheader()
        for item in list_dic:
            writer.writerow(item)

In [8]:
def getGNAndOption(file,seed):
    #利用network的
    fig = plt.figure(figsize=(100, 80))   #要在一开始调整图片大小，不然无法显示
    list_node,list_link,list_node_label,list_size=ReadNodeAndList(file)
    list_dic=[]
    #生成一个字典，后续还要更新就不放在其他函数里面了
    for i in range(len(list_node)):
        dic={}
        dic['id']=list_node[i]
        dic['label']=list_node_label[i]
        dic['size']=list_size[i]
        list_dic.append(dic)    
    G = nx.Graph()
    G.add_nodes_from(list_node)            #使用列表添加节点，但是如果要带其他属性则不方便
    G.add_weighted_edges_from(list_link)
    communities = nx.algorithms.community.girvan_newman(G)
    best_communities = None
    best_modularity = -1
    for community in communities: #获取最佳聚类结果
        #print(community)
        Q = modularity(G, community)  #用以衡量聚类效果的函数
        if Q > best_modularity:
            best_modularity = Q
            best_communities = community
        
    list_community=[]  #存所有簇    
    list_derail=[]  #存离群点
    for i, community in enumerate(best_communities):
        node=list(community)#字典转list
        if len(community)==1:   #对于离群点
            list_derail.append(node[0])
        if len(community)<=3 and len(community)>1:   #对于小群
            list_community.append(node)
        if len(community)>3:
            list_community.append(node)
    list_community.append(list_derail)    #最后一个一定是离群点集合    
    #print(list_community)
    list_typelabel=[]
    for type in range(len(list_community)):
        list_word=[]
        for item in list_community[type]:
            for index, word in enumerate(list_dic):  #找到对应id的label
                if word['id'] == item:
                  #print(word)
                  list_word.append(word['label'])
        #一个type得到一个标签
        typelabel=typeLabel(list_word)
        time.sleep(6)
        list_typelabel.append(typelabel)
    #装载如原字典
    for type in range(len(list_community)):    
        for item in list_community[type]:
            for index, word in enumerate(list_dic):  #找到对应id的label
                if word['id'] == item:
                  word['type']=type
                  word['typelabel']=list_typelabel[type]
    #for item in list_dic:
    #    print(item)
    return list_dic

In [9]:
def judgementWordInTopic(word,topic_exist):
    #本函数用以判断一个词是否已经存在于一个存在的话题中了
    #会人为的赋予一些经验和修正
    if(word=='elon' or word=='musk'):
        word='elonmusk'
    if(word in topic_exist):
        return 1
    if(word not in topic_exist):
        return 0    

In [10]:
def MergeTopic(list_all_topics,time_list):
    #合并各日期的话题
    #此函数得到的是一个全局性的总话题
    list_merge_topics=[]  #形如[[话题1单词],[话题2单词]]
    threshold=0.6       #合并话题的阈值
    for pos_day in range(len(list_all_topics)):
        #dic_merge_topics={}
        #dic_merge_topics['time']=time_list[pos_day]
        if(pos_day==0):  #第一天的数据直接加入话题
            for topic in list_all_topics[pos_day]:
                #print(topic)
                topic=topic['topic_word_list']  #只取话题中的词即可
                list_merge_topics.append(topic)
        else:
            for topic in list_all_topics[pos_day]:
                topic=topic['topic_word_list']  #只取话题中的词即可
                flag=0
                for topic_exist in list_merge_topics: #遍历已经存在的话题
                    count=0  #计数,若对于某一个已存在话题有多于百分之X的重合则视为同一个话题
                    for word in topic:
                        if judgementWordInTopic(word,topic_exist):
                            count+=1
                            flag=1
                    if(count/len(topic)>threshold):  #大于阈值则更新那一个
                        count_update_word=0  #更新词的pos
                        for word in topic:
                            if(judgementWordInTopic(word,topic_exist)==0):
                                topic_exist.append(word)
                                count_update_word+=1
                            if(count_update_word>2):  #只存前三个词
                                break
                        break                         
                if(flag==0):#表示作为一个新的话题
                    list_merge_topics.append(topic)
                    print('新话题')
    #for item in list_merge_topics:
    #    print(item)
    #到这一步 list_merge_topics已经完成了
    #之后包装一下，每个topics成字典   [[{代表词，序号,词}],[代表词，序号,词]]
    list_dic_merge_topics=[]
    for item in list_merge_topics:                 
         dic={}
         dic['type']=list_merge_topics.index(item)
         dic['word']=item
         dic['typelabel']=typeLabel(item)
         list_dic_merge_topics.append(dic)
    print(list_dic_merge_topics)            

In [11]:
def wordAccept(words,list_exist_all_topics,index,type):
    #用来判断本日的话题更新之后有哪些词
    global step
    words_add=[]
    if index<step:
        #在最开始的几天
        words_exist=[]
        for i in range (0,index):  #倒挡，把之前同一个话题的都加入
            for topic_exist in list_exist_all_topics[i]:
                if(topic_exist['type']==type):
                    words_exist+=topic_exist['word']
        words_add = words + [x for x in words if x not in words_exist]
        return words_add
    else:
        #最近一天的加入前70%,然后减半
        words_add=[]
        for i in range (1,step+1):  #倒挡，按比例把之前的加入
            for topic_exist in list_exist_all_topics[index-i]:
                if(topic_exist['type']==type):
                    length=int(len(topic_exist['word'])/(0.6/(2**index)))
                    #print(topic_exist['word'])
                    words_add+=topic_exist['word'][:length]  #顺序会被打乱
        words_add=words_add+words
        return list(set(words_add))     #去重后返回        

In [12]:
def duplicateRemovalType(list_exist):
    type_set = set()
    list_get=[]
    for item  in list_exist:
        if item['type'] not in type_set:  # 如果type没有出现过，则将其添加到type_set集合中
            type_set.add(item['type'])
            list_get.append(item)
        else:  # 如果type已经出现过，则找到之前的元素并进行合并
            for prev_item in list_get:
                if prev_item['type'] == item['type']:
                    prev_item['word']=prev_item['word']+item['word']
                    prev_item['word']=list(set(prev_item['word'])) 
                    break
    return list_get                

In [13]:
def mergeAndMinusTopic(list_alldays_topic,time_list):
    #合并各日期的topic,并且要求做到日期更新
    #输入的参数见getTopicChange
    list_exist_all_topics=[]   #里面每个list是一组（一组就是三天）的，每一组的中有数个{},每个{}表示一个话题的信息
    threshold=0.7
    type_max=0    #标记最靠后的一个type，比这个大的就是没有的topic
    #第一步，初始化一个话题。离群点已经去除
    list_exist=[]  #中间变量，某一组的话题
    for topic in list_alldays_topic[0]:   #把第一天的当成
        #print(topic)
        dic={}
        dic['type']=list_alldays_topic[0].index(topic)   #标记话题
        dic['word']=topic['topic_word_list']  #取话题中的词
        dic['time']=time_list[0]
        dic['size']=topic['size']
        type_max=dic['type']
        list_exist.append(dic)
    list_exist_all_topics.append(list_exist)
    #将之后的组更新入
    for index in range(1,len(time_list)):
        list_exist=[]
        print(time_list[index])
        for topic in list_alldays_topic[index]:
            words=topic['topic_word_list']
            flag=0  #用来标记新话题
            #print(words)
            for topic_exist in list_exist_all_topics[index-1]:  #倒退一组数据比话题,每个topic_exist是一个字典
                    count=0  #计数,若对于某一个已存在话题有多于百分之X的重合则视为同一个话题
                    for word in words:
                        count+=judgementWordInTopic(word,topic_exist['word'])  #在其中会返回1，不然就是返回0
                    if((count/len(words))>threshold):    #有一种可能情况时，两个子话题属于同一个已经存在的话题，要进行处理
                        flag=1  #表明不是新话题
                        dic={}
                        dic['type']=topic_exist['type']  #一样的话题就有一样的type
                        words_add=wordAccept(words,list_exist_all_topics,index,dic['type'])
                        dic['word']=words_add              #话题本次更新的词
                        dic['time']=time_list[index]
                        dic['size']=topic['size']
                        list_exist.append(dic)  
            if(flag==0):#表示作为一个新的话题
                dic={}
                type_max+=1
                dic['type']=type_max
                dic['word']=words
                dic['time']=time_list[index]
                dic['size']=topic['size']
                list_exist.append(dic)
        #有可能一个type出现两次，这里要进行合并。  之所以不在添加的时候判断，主要是为了节省运行时间
        list_exist = duplicateRemovalType(list_exist)
        list_exist_all_topics.append(list_exist)
    for topics in list_exist_all_topics:
        for topic in topics:
            print(topic)
    return   list_exist_all_topics
                    

In [14]:
def getTopicChange(file_list):
    #用这个函数把之前所有的node_type文件写入一个包含各个节点信息的csv
    #获取各日期type的改变
    #之后会调用merge合并各日期的话题
    list_all=[]
    #把所有的文件读取出来
    for file in file_list:
        list_file=[]
        with open(file+'-node_type.csv','r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                # 处理每行数据
                dic={}
                dic['label']=row[1]
                dic['size']=row[2]
                dic['type']=eval(row[3])
                dic['typelabel']=row[4]
                list_file.append(dic)
        list_all.append(list_file)  
    list_alldays_topic=[]
    for list_file in list_all:
        #print(list_file)
        list_topic=[]
        derail_topic = 0
        #找到离群的话题，去除
        for item in list_file:
            if(item['type']>derail_topic):
                derail_topic = item['type']
        #print(derail_topic)
        for item in list_file:
            #对于每一段时间的节点而言
            #print(item)
            if(item['type']!=derail_topic):
              if (judgementTypelabel(item['typelabel'],list_topic)==1): #判断字典中是否存在
                #存在则修改 
                for topic in list_topic:
                    if(topic['typelabel']==item['typelabel']):
                        topic['size']=topic['size']+eval(item['size'])  #更新权重
                        topic['topic_word_list'].append(item['label']) #更新词表
              else:
                 #不存在则加入
                dic={}
                dic['typelabel']=item['typelabel']
                dic['size']=eval(item['size'])
                dic['topic_word_list']=[item['label']]
                list_topic.append(dic)
        list_alldays_topic.append(list_topic)  #加入总的话题list
    #for topic in list_alldays_topic:
    #    print(file_list[list_alldays_topic.index(topic)])
    #    for item in topic:
    #        print(item)
    #list_alldays_topic中每个topic存储样式为
    #{'typelabel': 'openai', 'size': 2425.2642000000014, 'topic_word_list': ['ai', 'openai', 'microsoft', 'google', 'bing', 'bot', 'gpt3', 'msft', 'app', 'robot', 'siri']}
    #{'typelabel': 'chatbots', 'size': 525.2162000000001, 'topic_word_list': ['chatbot', 'chatbots', 'generative', 'language', 'technology', 'intelligence', 'bots']}
    #下面需要的是，整理合并具有相似内容的
    global time_list
    #MergeTopic(list_alldays_topic,time_list) 
    list_exist_all_topics = mergeAndMinusTopic(list_alldays_topic,time_list)
    return list_exist_all_topics

In [15]:
def judgementTypelabel(str,list_topic):
    #这个函数用来判断str(即typelabel)是否已经在字典中存在
    flag=0
    #print(list_topic)
    for dic in list_topic:
        if dic['typelabel']==str:
            flag=1
    return flag        

In [16]:
def pictureGephi(file_list,seed):
    #这个函数的作用是获取聚类结果并写入gephi所需的csv方便画图
    #只需要运行一次即可
    for file in file_list:
        list_dic=getGNAndOption(file,seed)
        writeTypeNode(file,list_dic)
        print('完成了一个')

In [17]:
def GetPointwiseMutualValue(word_1,word_2,doclist):
    value_1=0
    value_2=0
    value_mutual=0
    count_word=0  #一共出现多少词
    for item in doclist:
        if word_1 in item: value_1+=1
        if word_2 in item: value_2+=1
        if word_1 in item and word_2 in item: value_mutual+=1
        count_word=count_word+item.count(' ')+1 #用空格作为分词的依据
    return value_1/count_word,value_2/count_word,value_mutual/count_word

In [18]:
def predictTopic(todays_word,todays_weight,topics):
    ls_word=[]
    composition_number=1.66  #一个超参数,表示传入的话题数据需扩大多少。比如如果是一天的数据，在V3中（一组）就应该扩大3，若为两天的数据则应该扩大1.5
    model = models.word2vec.Word2Vec.load('word2vec.model')
    for word in todays_word:
        flag=0
        for topic in topics:
            if(word in topic['word']):
                flag=1
        if(flag==0):
            ls_word.append(word)
    #print(ls_word)                 #ls_word即为没有直接存在某一个话题中
    #1.首先，看看有没有可能的新话题生成
    list_newtopic=[]
    for i in range(len(ls_word)):
        for j in range(i,len(ls_word)):
            flag=0
            if(ls_word[i] !=ls_word[j]):
                    try:
                       similarity = model.wv.similarity(ls_word[i],ls_word[j])
                    except:
                       similarity=0   #主要是不存在某个词的情况
                    if(similarity>0.8):
                        if(len(list_newtopic)>0):
                            for item in list_newtopic:   #将新话题加入
                                if(ls_word[i] in item or ls_word[j] in item):
                                    item.append(ls_word[i])
                                    item.append(ls_word[j])
                                    item = list(set(item))
                                    flag=1
                        if(flag==0):
                            ls=[]
                            ls.append(ls_word[i])
                            ls.append(ls_word[j])        
                            list_newtopic.append(ls)
    #print(list_newtopic)
    
                            
    #2.然后，看看已有话题的影响
    baseline=0  #2.1计算每个topic的基准影响值，人话就是计算本来存在的话题本身单词的pmi均值,没用这个变量，直接算就行了。  放个变量只是为了表示有这么个东西
    for topic in topics:
        pos=0
        value_pmi_sum=0
        for i in range(len(topic['word'])):
            for j in range(i+1,len(topic['word'])):
                value_i,value_j,value_mutual=GetPointwiseMutualValue(topic['word'][i],topic['word'][j],doclist)
                if value_mutual != 0:
                    value_pmi=log((value_mutual/(value_i*value_j)),2)
                else:
                    value_pmi=0 
                if(value_pmi):    #话题增益
                    pos+=1
                    value_pmi_sum+=value_pmi
                    #print(topic['word'][i],topic['word'][j],value_pmi)
        if(pos!=0):            
            topic['pmi_baseline']= value_pmi_sum/pos
        else:
            topic['pmi_baseline']=0    
        #2.2计算未出现的词与话题的pmi   
    list_pmi=[]
    for word in ls_word:
        for topic in topics:
            pos=0
            value_pmi_sum=0
            #print(topic['pmi_baseline'])    
            for topic_word in topic['word']:
                value_i,value_j,value_mutual=GetPointwiseMutualValue(word,topic_word,doclist)
                if value_mutual != 0:
                    value_pmi=log((value_mutual/(value_i*value_j)),2)
                else:
                    value_pmi=0 
                if(value_pmi):    #话题增益
                        #print(word,topic['type'],value_pmi)
                        pos+=1
                        value_pmi_sum+=value_pmi
            if(pos>0):  #对于已经有增益的词            
                if((value_pmi_sum/pos)>topic['pmi_baseline']):        
                    dic={}
                    dic['word']=word
                    dic['type']=topic['type']
                    dic['pmi']=value_pmi_sum/pos
                    #print(dic)
                    list_pmi.append(dic)
    #print(list_pmi)
    #3. 计算新话题的PMI
    #具体计算
    for newtopic in list_newtopic:
        for i in range(len(newtopic)):
            for j in range(i+1,len(newtopic)): 
                #print(newtopic[i],newtopic[j])
                value_i,value_j,value_mutual=GetPointwiseMutualValue(newtopic[i],newtopic[j],doclist)
                if value_mutual != 0:
                    value_pmi=log((value_mutual/(value_i*value_j)),2)
                else:
                    value_pmi=0
                if(value_pmi): 
                    #print(newtopic[i],newtopic[j],value_pmi)          #暂时没想好怎么用
                    continue
    #3.1  看是否需要将新词加入
    #方法为计算一个不存在在某个话题中的词与话题中的词的相似度，有两个超过阈值的则加入
    for topic in topics:
        for word in ls_word:
            flag=0  #每个单词计算
            for topic_word in topic['word']:
                    try:
                       similarity = model.wv.similarity(word,topic_word)
                    except:
                       similarity=0   #主要是不存在某个词的情况
                    if(similarity>0.8):
                        flag+=1
            if(flag>=2):
                topic['word'].append(word)                           
                    
    #4.获取已有话题减益并归纳上面的结果
    result_get={}  #
    result_get['old_topic']=[]
    result_get['new_topic']=[]
    for topic in topics:
        dic={}
        now_size=0
        dic['type']=topic['type']
        for topic_word in topic['word']:
            if(topic_word in todays_word):
                  now_size=now_size+todays_weight[todays_word.index(topic_word)]
        if(now_size*composition_number>topic['size']*1.2):  #随便设一个超参数
            dic['change']='上升'
        if(now_size*composition_number<topic['size']*1.2 and now_size*composition_number>topic['size']*0.8):  #随便设一个超参数
            dic['change']='相对稳定'
        if(now_size*composition_number<topic['size']*0.8):  #随便设一个超参数
            dic['change']='下降'                  
        dic['word']=topic['word']
        dic['pre_size']=now_size*composition_number
        dic['time']=topic['time']
        dic['influence_word']=[]   #被影响的词
        for item in  list_pmi:
            if(item['type']==dic['type']):
                dic['influence_word'].append(item['word'])
        result_get['old_topic'].append(dic)                 
       # print(f"上一步size:{topic['size']},本次size:{now_size*3}")           
    result_get['new_topic']=list_newtopic        #不仅是新话题的出现 还要考虑新话题的排序   
    #for item in result_get['old_topic']:
    #    print(item)    
    #最终参数
    #list_pmi是本次新出现单词与以往type的影响
    #list_newtopic是获取的可能新话题
    #result_get是最终的结果，里面有两个属性，第一个属性是旧有话题的变化，第二个属性是新话题的产生
    return result_get
                      

In [19]:
#V3文件 2月5日前有一些小问题，就是数据经过了抽样（只取了11000多条数据，对实验结果影响不大。  暂时不改了
file_list=[
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月6日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月9日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月12日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月15日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月18日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月21日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月24日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月27日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/1月30日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月2日起11728全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月5日起22806全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月8日起24368全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月11日起17814全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月14日起18673全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月17日起16115全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月20日起17954全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月23日起15076全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/2月26日起13956全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月1日起17547全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月4日起13555全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月7日起15353全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月10日起12912全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月13日起25126全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月16日起25263全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月19日起20431全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月22日起24489全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月25日起19776全数据，边为相似度,门限0.8',
    'E:\code\舆情idea2\推特GPT\成图csvV3/3月28日起16429全数据，边为相似度,门限0.8',
]
time_list=[
    '1/6','1/9','1/12','1/15','1/18','1/21','1/24','1/27','1/30','2/2','2/5','2/8','2/11','2/14','2/17','2/20','2/23','2/26','3/1','3/4','3/7','3/10','3/13','3/16','3/19','3/22','3/25','3/28'
]
seed=1234
random.seed(seed)
step=2  #日期更新词的时候回顾的天数

In [20]:
#pictureGephi(file_list,seed)   #获得可用于gephi画图的_node_type csv
list_exist_all_topics = getTopicChange(file_list)       #获得话题,跑了一次存了就行,但是后面的代码需要这个值。  好在不花时间


1/9
1/12
1/15
1/18
1/21
1/24
1/27
1/30
2/2
2/5
2/8
2/11
2/14
2/17
2/20
2/23
2/26
3/1
3/4
3/7
3/10
3/13
3/16
3/19
3/22
3/25
3/28
{'type': 0, 'word': ['ai', 'openai', 'google', 'bing', 'microsoft', 'bot', 'gpt3', 'search'], 'time': '1/6', 'size': 1603.256100000001}
{'type': 1, 'word': ['chatbot', 'chatbots', 'language', 'generative', 'intelligence', 'technology'], 'time': '1/6', 'size': 362.46159999999986}
{'type': 2, 'word': ['chatgpt3', 'prompts', 'tweet', 'text', 'tweets', 'code', 'nlp'], 'time': '1/6', 'size': 200.7716}
{'type': 3, 'word': ['seo', 'content'], 'time': '1/6', 'size': 57.10700000000001}
{'type': 4, 'word': ['teachers', 'students', 'schools'], 'time': '1/6', 'size': 82.14900000000002}
{'type': 0, 'word': ['ai', 'openai', 'microsoft', 'google', 'bing', 'bot', 'gpt3', 'app', 'robot', 'app', 'robot'], 'time': '1/9', 'size': 1883.8482}
{'type': 1, 'word': ['chatbot', 'chatbots', 'generative', 'language', 'technology', 'intelligence'], 'time': '1/9', 'size': 316.4374999999999

In [21]:
def mergeTwoDic(dic_a,dic_b):
    #用来合并字典，字典中有word和weight
    #相同元素则更新值，否则加入
    #原始形式为{word:[q,q,w,w],weight:[1,1,2,2]}
    #print(dic_a)
    #先把a的词更新weight
    for word_a in dic_a['word']:
        for word_b in dic_b['word']:
            if word_a==word_b:
                dic_a['weight'][dic_a['word'].index(word_a)] = dic_a['weight'][dic_a['word'].index(word_a)]+ dic_b['weight'][dic_b['word'].index(word_b)]   
    #b中新词加入a
    for word_b in dic_b['word']:
        if(word_b not in dic_a['word']):
            dic_a['word'].append(word_b)
            dic_a['weight'].append(dic_b['weight'][dic_b['word'].index(word_b)])
    print(len(dic_a['word']))        
    return dic_a

In [22]:
day_verify=[
    #前后两个参数并不是一个日期。第二个参数是日期，第一个参数是所属的组
    #{'time':'2/14','day':'2023-02-15'},
    #{'time':'2/14','day':'2023-02-16'},
    #{'time':'2/17','day':'2023-02-17'},
    #{'time':'2/17','day':'2023-02-18'},
    {'time':'2/20','day':'2023-02-20'},
    {'time':'2/20','day':'2023-02-21'},
    {'time':'2/23','day':'2023-02-23'},
    {'time':'2/23','day':'2023-02-24'},
    {'time':'2/26','day':'2023-02-26'},
    {'time':'2/26','day':'2023-02-27'},
    {'time':'3/1','day':'2023-03-01'},
    {'time':'3/1','day':'2023-03-02'},
    {'time':'3/4','day':'2023-03-04'},
    {'time':'3/4','day':'2023-03-05'},
    {'time':'3/7','day':'2023-03-06'},
    {'time':'3/7','day':'2023-03-07'},
]
data_number=0   #表示取全部数据
list_temporary=[]
for i in range(0,len(day_verify),2):
    print(day_verify[i])
    pos_index=time_list.index(day_verify[i]['time'])
    topics=list_exist_all_topics[pos_index-1]  #要找上一次的
    data_number=0 
    dic_1,doclist_1 = GetKeyBertWord('GPT',day_verify[i]['day'])
    data_number=0 
    dic_2,doclist_2 = GetKeyBertWord('GPT',day_verify[i+1]['day'])
    dic=mergeTwoDic(dic_1,dic_2)
    doclist=doclist_1+doclist_2
    sorted_word = sorted(dic['word'], key=lambda x: dic['weight'][dic['word'].index(x)],reverse=True)  #根据weight操作word排序
    sorted_weight=sorted(dic['weight'],reverse=True)
    dic={}
    dic['sorted_word']=sorted_word[:50]
    dic['sorted_weight']=sorted_weight[:50]
    dic['topics']=topics
    list_temporary.append(dic) 

{'time': '2/20', 'day': '2023-02-20'}
总数量5948
取词中: 1.09983184799058347653 
总数量6079
取词中: 1.0 983547219480186464
8838
{'time': '2/23', 'day': '2023-02-23'}
总数量6018
取词中: 1.09983380422137285454 
总数量5135
取词中: 1.09980522010128563644 
8331
{'time': '2/26', 'day': '2023-02-26'}
总数量3373
取词中: 1.0997034400948992445 
总数量5212
取词中: 1.09980809825369415454 
6833
{'time': '3/1', 'day': '2023-03-01'}
总数量5751
取词中: 1.09982608695652185656 
总数量6402
取词中: 1.0 984377441024847666
8598
{'time': '3/4', 'day': '2023-03-04'}
总数量4144
取词中: 1.0 97586290127927345
总数量4149
取词中: 1.09975891996142727656 
7034
{'time': '3/7', 'day': '2023-03-06'}
总数量5265
取词中: 1.09981003039513685557 
总数量5132
取词中: 1.09980510621711176556 
8098


In [26]:
for data in list_temporary:
    result_get = predictTopic(data['sorted_word'],data['sorted_weight'],data['topics'])
    print(result_get)

{'old_topic': [{'type': 0, 'change': '上升', 'word': ['gpt3', 'azure', 'bot', 'ai', 'chatgpt3', 'app', 'api', 'openai', 'robot', 'search', 'msft', 'siri', 'bing', 'google', 'gpt4', 'microsoft', 'bard'], 'pre_size': 2813.579484000001, 'time': '2/17', 'influence_word': ['generativeai', 'openaichatgpt', 'machinelearning', 'controversial', 'advertising', 'aipad']}, {'type': 1, 'change': '上升', 'word': ['technology', 'intelligence', 'chatbot', 'robots', 'generative', 'chatbots', 'conversational', 'language'], 'pre_size': 688.056222, 'time': '2/17', 'influence_word': ['artificialintelligence', 'generativeai', 'openaichatgpt', 'twitter', 'machinelearning', 'controversial', 'advertising', 'aipad']}, {'type': 31, 'change': '相对稳定', 'word': ['chatgpt3', 'elonmusk', 'opera', 'elon', 'musk', 'memecoin', 'https'], 'pre_size': 193.43465399999997, 'time': '2/17', 'influence_word': ['openaichatgpt', 'controversial']}], 'new_topic': [['trading', 'crypto', 'crypto', 'web3'], ['marketing', 'advertising'], ['

In [25]:

for topics_day in list_exist_all_topics:
    for topic in topics_day:
        print(topic)

{'type': 0, 'word': ['ai', 'openai', 'google', 'bing', 'microsoft', 'bot', 'gpt3', 'search'], 'time': '1/6', 'size': 1603.256100000001}
{'type': 1, 'word': ['chatbot', 'chatbots', 'language', 'generative', 'intelligence', 'technology'], 'time': '1/6', 'size': 362.46159999999986}
{'type': 2, 'word': ['chatgpt3', 'prompts', 'tweet', 'text', 'tweets', 'code', 'nlp'], 'time': '1/6', 'size': 200.7716}
{'type': 3, 'word': ['seo', 'content'], 'time': '1/6', 'size': 57.10700000000001}
{'type': 4, 'word': ['teachers', 'students', 'schools'], 'time': '1/6', 'size': 82.14900000000002}
{'type': 0, 'word': ['ai', 'openai', 'microsoft', 'google', 'bing', 'bot', 'gpt3', 'app', 'robot', 'app', 'robot'], 'time': '1/9', 'size': 1883.8482}
{'type': 1, 'word': ['chatbot', 'chatbots', 'generative', 'language', 'technology', 'intelligence'], 'time': '1/9', 'size': 316.4374999999999}
{'type': 5, 'word': ['chatgpt3', 'msft', 'nlp', 'text', 'prompts', 'https', 'elonmusk', 'code'], 'time': '1/9', 'size': 211.54